In [ ]:
! pip install folium pandas

In [ ]:
import folium
import re
import pandas as pd

In [ ]:
geocoded_exposure_sites = pd.read_parquet("geocoded_exposure_sites.parquet")
geocoded_exposure_sites.columns = geocoded_exposure_sites.columns.str.strip().str.lower().str.replace('[()]', '', regex=True).str.replace('[^0-9a-zA-Z_]', '_', regex=True)
geocoded_exposure_sites = geocoded_exposure_sites.assign(tier = lambda x: x.advice_title.str.extract("Tier ([0-9])"))

In [ ]:
m = folium.Map([-37.813611, 144.963056]) # centre of Melbourne
for site in geocoded_exposure_sites[geocoded_exposure_sites.geometry_lng.isna() == False].itertuples():
    tooltip = site.site_title
    popup = f"""<h3>{site.site_title}</h3><br>
        {site.advice_title}<br>
        {site.exposure_date}<br>
        {site.exposure_time_start_24} to {site.exposure_time_end_24}<br>
        {site.advice_instruction}"""
    colour = "darkred" if site.tier=="1" else "red" if site.tier=="2" else "orange"
    folium.CircleMarker(location=[site.geometry_lat, site.geometry_lng],popup=folium.Popup(popup, max_width=800),tooltip=tooltip,color=colour, radius=5).add_to(m)
    #folium.Marker(location=[site.geometry_lat, site.geometry_lng],popup=folium.Popup(popup, max_width=800),tooltip=tooltip,icon=folium.Icon(color=colour, icon='exclamation', prefix='fa')).add_to(m)
m